# Market Feature Table

## 1. Objective

- The Client ComZ is an ecommerce company. The company wants to focus on targeting the right customers  with the right products to increase overall revenue and conversion rate.

- To target the right customers with the right products, they need to build an ML model for marketing based on user interaction with products in the past like number of views,  most viewed product, number of activities of user, vintage of user and others. 

- ComZ has contacted the Data Science and Engineering team to use this information to fuel the personalized advertisements, email marketing campaigns, or special offers on the landing and category pages of the company's website.

## 2. Data Description

**1. Visitor Log Data** – It is a browsing log data of all the visitors and the users. This table contains the following information:

- *WebClientID*:
Unique ID of browser for every system. (If a visitor is using multiple browsers on a system like Chrome, Safari, then there would be a different web clientid for each browser). The ID remains consistent unless the user clears their cookie.

- *VisitDateTime*:
Date and time of visit. There are two different formats for DateTime. 
One is in datetime format “2018-05-07 04:28:45.970”
Another one is in unix datetime format “1527051855673000000”

- *ProductID*:
Unique ID of product browsed/ clicked by the visitor

- *UserID*:     
Unique ID of the registered user. As expected, this is available for registered users only, not for all visitors. 

- *Activity*:
Type of activity can be browsing (pageload) or clicking a product

- *Browser*:
Browser used by the visitor

- *OS*:
Operating System of the system used by the visitor

- *City*:
City of the visitor

- *Country*:
Country of the visitor


**2. User Data** – It has registered user information like signup date and segment.

- *UserID*:
Unique ID of the registered user.

- *Signup Date*:
Date of registration for the user

- *User Segment*:
User Segment (A/B/C) created based on historical engagement

In [1]:
# importing packages
import numpy as np
import pandas as pd
import os
import datetime
import time
import warnings

from datetime import datetime, timezone
from difflib import SequenceMatcher

# setting directory and pandas max rows/cols
os.chdir("D:\\EDA\\JobAThon\\ecommerce")
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

warnings.filterwarnings("ignore")

In [2]:
# importing data files
userTable_data = pd.read_csv("userTable.csv")
VisitorLogs_data = pd.read_csv("VisitorLogsData.csv")

In [3]:
#looking rows and columns are in both files
print("the number of rows and columns in the userTable are:{} ".format(userTable_data.shape))
print("the number of rows and columns in the VisitorLogs_data are:{} ".format(VisitorLogs_data.shape))


the number of rows and columns in the userTable are:(34050, 3) 
the number of rows and columns in the VisitorLogs_data are:(6588000, 9) 


In [4]:
#looking first 2 rows in both files
print(userTable_data.head(2))
print("\n")
print("******"*10)
print(VisitorLogs_data.head(2))

    UserID                       Signup Date User Segment
0  U133159  2018-04-14 07:01:16.202607+00:00            C
1  U129368  2017-12-02 09:38:41.584270+00:00            B


************************************************************
     webClientID            VisitDateTime ProductID UserID Activity  \
0  WI10000050298  2018-05-07 04:28:45.970  pr100631    NaN      NaN   
1  WI10000025922  2018-05-13 07:26:04.964  pr100707    NaN      NaN   

         Browser       OS     City Country  
0  Chrome Mobile  Android  Chennai   India  
1         Chrome  Windows      NaN  Taiwan  


In [5]:
# userTable datetime
userTable_data["Signup Date"] = pd.to_datetime(userTable_data["Signup Date"])

In [6]:
userTable_data["Signup Date"].head()

0   2018-04-14 07:01:16.202607+00:00
1   2017-12-02 09:38:41.584270+00:00
2          2013-03-19 11:38:55+00:00
3   2018-01-18 08:29:51.627954+00:00
4   2018-03-27 08:05:28.806800+00:00
Name: Signup Date, dtype: datetime64[ns, UTC]

In [26]:
# visitorlogsData converting to uppercase
VisitorLogs_data.webClientID = VisitorLogs_data.webClientID.str.upper()
VisitorLogs_data.ProductID = VisitorLogs_data.ProductID.str.upper()
VisitorLogs_data.UserID = VisitorLogs_data.UserID.str.upper()
VisitorLogs_data.Activity = VisitorLogs_data.Activity.str.upper()
VisitorLogs_data.Browser = VisitorLogs_data.Browser.str.upper()
VisitorLogs_data.OS = VisitorLogs_data.OS.str.upper()
VisitorLogs_data.City = VisitorLogs_data.City.str.upper()
VisitorLogs_data.Country = VisitorLogs_data.Country.str.upper()

In [27]:
## VisitDateTime - separating into unix vs non-unix dates

VisitorLogs_data['date_time_format'] = VisitorLogs_data['VisitDateTime'].str.isdigit().fillna(False)

date_time_format1 = VisitorLogs_data[VisitorLogs_data['date_time_format']==True]
date_time_format2 = VisitorLogs_data[VisitorLogs_data['date_time_format']==False]

date_time_format1.VisitDateTime = pd.to_numeric(date_time_format1.VisitDateTime)
date_time_format1['temp_VisitDateTime'] = (date_time_format1.VisitDateTime/1000000)
date_time_format1['temp_VisitDateTime'] = date_time_format1['temp_VisitDateTime'].apply(np.int64)
date_time_format1['VisitDateTime'] =  pd.to_datetime(date_time_format1['temp_VisitDateTime'], unit='ms')
date_time_format1 = date_time_format1.drop(columns = 'temp_VisitDateTime')

date_time_format2.VisitDateTime = pd.to_datetime(date_time_format2.VisitDateTime)


In [28]:
## appending converted datetimes dataframes
new_VLogsData = date_time_format1.append(date_time_format2)
new_VLogsData = new_VLogsData.drop(columns = 'date_time_format')
new_VLogsData['date'] = pd.DatetimeIndex(new_VLogsData.VisitDateTime).date

In [29]:
## missing value treatments / imputations for Activity and ProductID
new_VLogsData = new_VLogsData.sort_values(['UserID','webClientID','VisitDateTime'])

new_VLogsData['oldProductID'] = new_VLogsData.groupby(['UserID','date'])['ProductID'].shift(1)
new_VLogsData['priorProductID'] = new_VLogsData.groupby(['UserID'])['ProductID'].shift(1)
new_VLogsData['nextProductID'] = new_VLogsData.groupby(['UserID'])['ProductID'].shift(-1)

new_VLogsData['newActivity'] = np.where(new_VLogsData['ProductID']==new_VLogsData['oldProductID'],'CLICK','PAGELOAD')
new_VLogsData['Activity'] = np.where(new_VLogsData['Activity'].notnull(),new_VLogsData['Activity'],new_VLogsData['newActivity'])

new_VLogsData['newProductID'] = np.where(new_VLogsData['Activity']=='CLICK',new_VLogsData['priorProductID'],new_VLogsData['nextProductID'])
new_VLogsData['ProductID'] = np.where(new_VLogsData['ProductID'].notnull(),new_VLogsData['ProductID'],new_VLogsData['newProductID'])

#Null values are filled with forward filling and backward filling 
new_VLogsData['ProductID'] = np.where(new_VLogsData['ProductID'].notnull(),new_VLogsData['ProductID'],new_VLogsData['ProductID'].ffill().bfill())

new_VLogsData['priordate'] = new_VLogsData.groupby(['UserID'])['date'].shift(1)
new_VLogsData['nextdate'] = new_VLogsData.groupby(['UserID'])['date'].shift(-1)
new_VLogsData = new_VLogsData.sort_values(['UserID','webClientID','ProductID'])
new_VLogsData['VisitDateTime'] = np.where(new_VLogsData['VisitDateTime'].notnull(),new_VLogsData['VisitDateTime'],new_VLogsData['VisitDateTime'].ffill().bfill())


In [30]:
# dropping duplicate records by user/system and time

new_VLogsData = new_VLogsData.drop_duplicates(subset = ['UserID', 'webClientID','VisitDateTime'], keep = 'first').reset_index(drop = True)


In [31]:
# calculation

last_7_days = new_VLogsData[new_VLogsData['VisitDateTime']>=(max(new_VLogsData['VisitDateTime'])-pd.to_timedelta(7, unit='d'))]
last_15_days = new_VLogsData[new_VLogsData['VisitDateTime']>=(max(new_VLogsData['VisitDateTime'])-pd.to_timedelta(15, unit='d'))]

# No_of_days_Visited_7_Days
# How many days a user was active on platform in the last 7 days - count distinct visitdatetime by userid when webclientid = 1
no_of_visits_7_days = pd.DataFrame(last_7_days.groupby('UserID').date.nunique())
no_of_visits_7_days = no_of_visits_7_days.reset_index()
no_of_visits_7_days = no_of_visits_7_days.rename(columns = {'date':'No_of_days_Visited_7_Days'})

# Clicks_last_7_days - Count of Clicks in the last 7 days  by the user
click_counts = pd.DataFrame(last_7_days[last_7_days['Activity']=='CLICK'].groupby('UserID').Activity.count())
click_counts = click_counts.reset_index()
click_counts = click_counts.rename(columns = {'Activity':'Clicks_last_7_days'})

# Pageloads_last_7_days - Count of pageloads in the last 7 days  by the user
pageload_counts = pd.DataFrame(last_7_days[last_7_days['Activity']=='PAGELOAD'].groupby('UserID').Activity.count())
pageload_counts = pageload_counts.reset_index()
pageload_counts = pageload_counts.rename(columns = {'Activity':'Pageloads_last_7_days'})

# No_Of_Products_Viewed_15_Days - Number of Products viewed by the user in the last 15 days
no_prod_15d = pd.DataFrame(last_15_days.groupby('UserID').ProductID.nunique())
no_prod_15d = no_prod_15d.reset_index()
no_prod_15d = no_prod_15d.rename(columns = {'ProductID':'No_Of_Products_Viewed_15_Days'})

In [32]:
# Most frequently viewed (page loads) product by the user in the last 15 days. If there are multiple products that have 
# a similar number of page loads then , consider the recent one. If a user has not viewed any product in the last 15 days 
# then put it as Product101. 

most_view_prod_15d = last_15_days[last_15_days['Activity']=='PAGELOAD'].groupby(['UserID','ProductID']).\
                                agg({'Activity':'count','VisitDateTime':'max'})
most_view_prod_15d = most_view_prod_15d.reset_index()

most_view_prod_15d_grp = most_view_prod_15d.sort_values(['Activity','VisitDateTime'], ascending=False).drop_duplicates(['UserID'])
most_view_prod_15d_grp =most_view_prod_15d_grp.sort_values('UserID')

most_view_prod = most_view_prod_15d_grp[['UserID','ProductID']]
most_view_prod = most_view_prod.rename(columns = {'ProductID':'Most_Viewed_product_15_Days'})

In [34]:
#as per the datset given time is calculated from 2018-05-27
#User vintage is today - signup date (Vintage (In Days) of the user as of today)
user_vintage_days = userTable_data.copy()
user_vintage_days['today'] = pd.to_datetime('2018-05-27',utc=True)
user_vintage_days['User_Vintage'] = (user_vintage_days['today']-user_vintage_days['Signup Date']).dt.days
user_vintage_days= user_vintage_days[['UserID','User_Vintage']]


In [35]:
# Most_Active_OS - Most Frequently used OS by user
most_active_os_data = new_VLogsData.groupby(['UserID','OS']).\
                                agg({'webClientID':'count',\
                                    'VisitDateTime':'max'})
most_active_os_data = most_active_os_data.sort_values(by = ['UserID']).reset_index()

most_active_os_data_grp = most_active_os_data.sort_values(['webClientID','VisitDateTime'], ascending=False).drop_duplicates(['UserID'])

most_active_os_df = most_active_os_data_grp[['UserID','OS']]
most_active_os_df = most_active_os_df.rename(columns = {'OS':'Most_Active_OS'})
most_active_os_df = most_active_os_df.sort_values('UserID')




In [36]:
# Recently_Viewed_Product - Most recently viewed (page loads) product by the user. If a user has not viewed any product then put it as Product101.

recent_product = new_VLogsData[(new_VLogsData['Activity']=='PAGELOAD') & (pd.notnull(new_VLogsData['ProductID']))]
recent_product = recent_product[['UserID','ProductID','VisitDateTime']]

recent_product_grp = recent_product.sort_values('VisitDateTime', ascending=False).drop_duplicates(['UserID'])
recent_product_grp = recent_product_grp.sort_values('UserID')
recent_product_grp = recent_product_grp[['UserID','ProductID']]
recent_product_grp = recent_product_grp.rename(columns = {'ProductID':'Recently_Viewed_Product'})



In [38]:
# Merging total data

main_data = userTable_data[['UserID']]

join_newVLogsData = main_data\
            .merge(no_of_visits_7_days, how='left', on = 'UserID')\
            .merge(no_prod_15d, how='left', on = 'UserID')\
            .merge(user_vintage_days, how='left', on = 'UserID')\
            .merge(most_view_prod, how='left', on = 'UserID')\
            .merge(most_active_os_df, how='left', on = 'UserID')\
            .merge(recent_product_grp, how='left', on = 'UserID')\
            .merge(pageload_counts, how='left', on = 'UserID')\
            .merge(click_counts, how='left', on = 'UserID')


In [39]:
# Missing value treatment by adding Product101

join_newVLogsData['No_of_days_Visited_7_Days'] = join_newVLogsData['No_of_days_Visited_7_Days'].replace(np.nan, 0)
join_newVLogsData['No_Of_Products_Viewed_15_Days'] = join_newVLogsData['No_Of_Products_Viewed_15_Days'].replace(np.nan, 0)
join_newVLogsData['User_Vintage'] = join_newVLogsData['User_Vintage'].replace(np.nan, 0)
join_newVLogsData['Pageloads_last_7_days'] = join_newVLogsData['Pageloads_last_7_days'].replace(np.nan, 0)
join_newVLogsData['Clicks_last_7_days'] = join_newVLogsData['Clicks_last_7_days'].replace(np.nan, 0)
join_newVLogsData['Most_Viewed_product_15_Days'] = join_newVLogsData['Most_Viewed_product_15_Days'].replace(np.nan, 'PRODUCT101')
join_newVLogsData['Recently_Viewed_Product'] = join_newVLogsData['Recently_Viewed_Product'].replace(np.nan, 'PRODUCT101')


In [40]:
# changing datatypes to int type

join_newVLogsData['No_of_days_Visited_7_Days'] = join_newVLogsData['No_of_days_Visited_7_Days'].astype(int)
join_newVLogsData['No_Of_Products_Viewed_15_Days'] = join_newVLogsData['No_Of_Products_Viewed_15_Days'].astype(int)
join_newVLogsData['User_Vintage'] = join_newVLogsData['User_Vintage'].astype(int)
join_newVLogsData['Pageloads_last_7_days'] = join_newVLogsData['Pageloads_last_7_days'].astype(int)
join_newVLogsData['Clicks_last_7_days'] = join_newVLogsData['Clicks_last_7_days'].astype(int)


In [41]:
#sorting data with based on UserID
join_newVLogsData.sort_values('UserID',inplace=True)

# exporting final to csv
join_newVLogsData.to_csv("my_submission_file.csv",index=False)